In [ ]:
# # Colab set up
# !git clone https://github.com/minggli/mnist-gan.git -b dev

In [ ]:
# import os
# os.chdir('./mnist-gan')
# !git clone https://github.com/tensorflow/models.git
# !pip install -r requirements.txt

In [ ]:
import tensorflow as tf

from pipeline import mnist_batch_iter
from output import produce_grid, produce_gif
from graph import *

In [ ]:
sess = tf.Session()
init_op = tf.global_variables_initializer()

sess.run(init_op)

for var in tf.trainable_variables():
    print(var)

grids_through_epochs = list()
constant = g.gaussian_noise(g_x)

In [ ]:
for epoch in range(1, EPOCH + 1):
    try:
        step = 0
        sess.run(mnist_batch_iter.initializer)
        while True:
            try:
                for i in range(N_CRITIC):
                    step += 1
                    _, d_loss_score = sess.run(
                                        fetches=[d_train_step, d_loss],
                                        feed_dict={g_x: g.gaussian_noise(g_x)})
                    print("Epoch {0} of {1}, step {2} "
                          "Discriminator log loss {3:.4f}".format(
                            epoch, EPOCH, step, d_loss_score))

                step += 1
                _, g_loss_score = sess.run(
                                        fetches=[g_train_step, g_loss],
                                        feed_dict={g_x: g.gaussian_noise(g_x)})
                print("Epoch {0} of {1}, step {2}"
                      " Generator log loss {3:.4f}".format(
                        epoch, EPOCH, step, g_loss_score))

            except tf.errors.OutOfRangeError:
                print("Epoch {0} has finished.".format(epoch))
                break
    except KeyboardInterrupt:
        print("Ending Training during {0} epoch.".format(epoch))
        break
    test_images = sess.run(g_o, feed_dict={g_x: constant, is_train: False})
    grids_through_epochs.append(
        produce_grid(test_images, epoch, './results', save=True))

produce_gif(grids_through_epochs, path='./results')